# Hidden Markov Models

## Problem 1
- Building HMM and generating samples

In [1]:
import numpy as np
from hmmlearn import hmm
np.random.seed(42)

In [24]:
model = hmm.GaussianHMM(n_components=3, covariance_type="full")

In [25]:
model.startprob_ = np.array([0.6, 0.3, 0.1])

In [26]:
model.transmat_ = np.array([[0.7,0.2,0.1],
                            [0.3,0.5,0.2],
                            [0.3,0.3,0.4]])

In [27]:
model.means_ = np.array([[0,0],[3,-3],[5,10]])

In [28]:
model.covars_ = np.tile(np.identity(2), (3,1,1))

In [29]:
X, Z = model.sample(100)

In [12]:
lr = hmm.GaussianHMM(n_components=3, covariance_type="diag",
                     init_params="cm", params="cmt")

In [13]:
lr.startprob_ = np.array([1.0, 0.0, 0.0])

In [14]:
lr.transmat_ = np.array([[0.5, 0.5, 0.0],
                         [0.0, 0.5, 0.5],
                         [0.0, 0.0, 1.0]])

- Fixing parameters

In [16]:
model = hmm.GaussianHMM(n_components=3, n_iter=100, init_params="mcs")
model.transmat_ = np.array([[0.7, 0.2, 0.1],
                            [0.3, 0.5, 0.2],
                            [0.3, 0.3, 0.4]])

- Training HMM parameters and inferring the hidden states

In [38]:
remodel = hmm.GaussianHMM(n_components=3, covariance_type="full", n_iter=1000)
remodel.fit(X)
Z2 = remodel.predict(X)

In [46]:
(Z2==Z).sum()/len(Z)

0.81

In [45]:
remodel.transmat_

array([[9.23599049e-01, 4.77474640e-04, 7.59234766e-02],
       [1.52620393e-01, 7.53643125e-01, 9.37364820e-02],
       [3.29080124e-01, 3.07283512e-01, 3.63636364e-01]])

In [42]:
model.transmat_

array([[0.7, 0.2, 0.1],
       [0.3, 0.5, 0.2],
       [0.3, 0.3, 0.4]])

- Monitoring convergence

In [36]:
remodel.monitor_

ConvergenceMonitor(
    history=[-664.5662350121705, -439.7897867406189, -429.6481692598469, -418.54422777970524, -408.5574161037446, -400.38940733276667, -394.90771144633027, -391.1513021967038, -388.511544288925, -385.594397798375, -376.39385063487333, -369.7560560899268, -368.88038133929746, -368.67583590214895, -368.58428363001707, -368.50676581790754, -368.4260171189118, -368.34219078407074, -368.26281394685213, -368.19572272370556, -368.14491960997873, -368.11005932072595, -368.08804589650674, -368.07502548357934, -368.06768838511937],
    iter=25,
    n_iter=1000,
    tol=0.01,
    verbose=False,
)

In [37]:
remodel.monitor_.converged

True

- Working with multiple sequences

In [47]:
X1 = [[0.5], [1.0], [-1.0], [0.42], [0.24]]

In [48]:
X2 = [[2.4], [4.2], [0.5], [-0.24]]

In [49]:
X = np.concatenate([X1, X2])

In [50]:
lengths = [len(X1), len(X2)]

In [53]:
hmm.GaussianHMM(n_components=3).fit(X, lengths)

Fitting a model with 14 free scalar parameters with only 9 data points will result in a degenerate solution.


GaussianHMM(n_components=3)

## Problem 2 - The Dishonest Casino

First the data will be loaded

In [153]:
import numpy as np
from hmmlearn import hmm
from sklearn.preprocessing import LabelEncoder

In [154]:
with open("rolls.txt", "r") as f:
    X = list(
        map(lambda x: int(x.strip("\n")), f.readlines())
    )
    X = np.array(X).reshape(-1,1)-1
    f.close()

with open("dice.txt", "r") as f:
    y = list(map(lambda x: x.strip("\n"), f.readlines()))
    #y = np.array(y).reshape(-1,1)
    enc = LabelEncoder().fit(y)
    y = enc.transform(y).reshape(-1,1)

In [256]:
def train_test_split( arrays: list, test_size):
    out = []
    for arr in arrays:
        arr_train, arr_test = arr[:test_size], arr[test_size:]
        out.append(arr_train)
        out.append(arr_test)
    return out


### a) construct the model assuming all parameters known.

In [260]:
X_train, X_test, y_train, y_test = train_test_split((X, y), -10)

- Creation of model with all known parameters

In [261]:
model = hmm.CategoricalHMM(n_components=2, init_params="mc")
model.n_features=6
model.transmat_ = np.array([[0.95, 0.05],
                            [0.1, 0.9]])
model.emissionprob_ = np.array([[1/6]*6,
                                [1/10]*5 + [5/10]])
model.startprob_ = np.array([1.0, 0.0])

In [262]:
model.score(X_test)

-18.228865102347154

- Creation of model with unknown transition probability matrix

In [284]:
remodel = hmm.CategoricalHMM(n_components=2, init_params="mt", random_state=22)
remodel.n_features=6

remodel.emissionprob_ = np.array([[1/6]*6,
                                [1/10]*5 + [5/10]])
remodel.startprob_ = np.array([1.0, 0.0])

In [285]:
remodel.fit(X_train)
remodel.monitor_.converged

True

- Comparison of real and estimated transition matrix

In [289]:
remodel.transmat_

array([[0.24528262, 0.75471738],
       [0.30274841, 0.69725159]])

In [242]:
model.transmat_

array([[0.95, 0.05],
       [0.1 , 0.9 ]])

The model score was similar to the model that had all parameters kown.<br>
However, the transition matrix obtained was not like the original.<br>

- Comparison of the generated dice series **VS** the original dice series

In [307]:
dice_series_generated, dice_state = remodel.sample(10, random_state=22)

In [309]:
dict(zip(*np.unique(dice_series_generated, return_counts=True)))

{0: 1, 1: 1, 2: 2, 3: 1, 4: 1, 5: 4}

In [310]:
dict(zip(*np.unique(X_test, return_counts=True)))

{0: 1, 1: 1, 2: 4, 3: 1, 4: 2, 5: 1}

In [311]:
print(np.concatenate((dice_state.reshape(-1,1), y_test), axis=1))

[[0 0]
 [1 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [1 0]
 [1 0]]


We can see that the frequencies of the Validation set are uniform. <br>
The frequency of the generated sample is not, has it is more biased towards the value 5, which maps into the dice roll of 6. 

Besides that we can see that the state changed with a lot of frequency, in contrast to the original dice historical, which explains why the frequency of the generated samples are not uniformly distributed.

----

### c) Train HMM’s with half of the data points and observe the log-likelihood with the remaining half. 
- Compare the estimated parameters (transition and emission matrices) with the original model from question a);
- Visualize the generated dice series of the last half of data against the given one. (Note: Use the dataset provided in files rolls.txt and dice.txt provided on Moodle);

In [313]:
N = X.shape[0]//2

In [314]:
X_train, X_test, y_train, y_test = train_test_split((X, y), N)

####    1. Assume both transition and emission matrices are unknown

In [315]:
remodel = hmm.CategoricalHMM(n_components=2, random_state=22, n_iter=1000, init_params="te")
remodel.n_features=6

remodel.fit(X_train)

CategoricalHMM(init_params='te', n_components=2, n_iter=1000,
               random_state=RandomState(MT19937) at 0x253BF5EC440)

In [316]:
remodel.score(X_test)

-972.3959556320895

In [317]:
model.score(X_test)

-961.4177962080355

Like before, the log-likelihood scores are not very distinct from each other. <br>
The original model performed better